In [1]:
import numpy as np

from pprint import pprint
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cosine

import tensorflow as tf

In [2]:
N = 2000
dim = 5
theta = np.pi/8
np.random.seed(0)

vecs = np.random.rand(N, dim)
dists = cosine_similarity(vecs, vecs)
cond = dists > np.cos(theta)
ixs = np.triu_indices(N, k=1)

#################################################################

data = np.hstack((vecs[ixs[0]], vecs[ixs[1]]))
#data = np.subtract(vecs[ixs[0]],  vecs[ixs[1]])
#data = np.reshape(1 - dists[ixs], (len(ixs[0]), -1))

#################################################################

data_y = cond[ixs].astype(int)
un = np.unique(data_y, return_counts=True)
print(un)
counts = un[1]
print('baseline: %s\n' % (counts[0]/sum(counts)))

X_train, X_test, y_train, y_test = train_test_split(
    data, data_y, test_size=0.33, random_state=42, stratify=data_y)

if data.shape[1] > 1:
    norm = StandardScaler()
    X_train = norm.fit_transform(X_train)
    X_test = norm.transform(X_test)

def gen_batches(X, y, batch_size=100):
    for i in range(0, X.shape[0], batch_size):
        yield X[i:i+batch_size], y[i:i+batch_size]


(array([0, 1]), array([1765742,  233258]))
baseline: 0.8833126563281641



In [3]:
learning_rate = 0.001
batch_size = 128
num_steps = int(X_train.shape[0]/batch_size) + 1

In [4]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=data.shape[1])]

# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[128, 128],
    optimizer=tf.train.AdamOptimizer(learning_rate),
    n_classes=2,
    loss_reduction=tf.losses.Reduction.MEAN,
    dropout=0.1,
    #model_dir="./tmp/dnn_model"
)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    num_epochs=1,
    batch_size=batch_size,
    shuffle=True 
)

classifier.train(input_fn=train_input_fn, steps=num_steps)

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
test_eval = classifier.evaluate(input_fn=test_input_fn)
print("\ntest metrics:\n")
pprint(test_eval)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/lm/6pdtshnd68xbxpblh7xytq6m0000gp/T/tmpwhpc4nmx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f08bda0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for 

INFO:tensorflow:global_step/sec: 626.632
INFO:tensorflow:loss = 0.10217313, step = 5901 (0.160 sec)
INFO:tensorflow:global_step/sec: 648.371
INFO:tensorflow:loss = 0.0389708, step = 6001 (0.154 sec)
INFO:tensorflow:global_step/sec: 695.309
INFO:tensorflow:loss = 0.06640726, step = 6101 (0.144 sec)
INFO:tensorflow:global_step/sec: 719.72
INFO:tensorflow:loss = 0.051749185, step = 6201 (0.139 sec)
INFO:tensorflow:global_step/sec: 718.789
INFO:tensorflow:loss = 0.08423976, step = 6301 (0.139 sec)
INFO:tensorflow:global_step/sec: 682.634
INFO:tensorflow:loss = 0.033098985, step = 6401 (0.147 sec)
INFO:tensorflow:global_step/sec: 686.672
INFO:tensorflow:loss = 0.025410194, step = 6501 (0.146 sec)
INFO:tensorflow:global_step/sec: 734.917
INFO:tensorflow:loss = 0.037050933, step = 6601 (0.136 sec)
INFO:tensorflow:global_step/sec: 726.811
INFO:tensorflow:loss = 0.064977095, step = 6701 (0.138 sec)
INFO:tensorflow:global_step/sec: 692.209
INFO:tensorflow:loss = 0.05270689, step = 6801 (0.144 se